<a href="https://colab.research.google.com/github/dvqthinh25111999/video_summarize_dsf/blob/master/Copy_of_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pwd
import os
os.chdir('/content/drive/My Drive/video_summarize_dsf')

!pip install chainer
!pip install torch
!pip install scikit-video

/content


In [3]:
import numpy as np
import os
import json

from func.nets import vid_enc, vid_enc_vgg19
from chainer import serializers

import sys
sys.path.append('script/')
from summarize import get_flabel
from func.sampling.vsum import VSUM

import chainer
from chainer import configuration

import torch

/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))


In [4]:
# settings
feat_type = 'smt_feat' # smt_feat (proposed) or vgg

# load embedding model
if feat_type == 'smt_feat':
    model = vid_enc.Model()
    print('model:', str(model))
    serializers.load_npz('data/trained_model/model_par', model)
    print('model:', str(model))
elif feat_type == 'vgg':
    model = vid_enc_vgg19.Model()
else:
    raise RuntimeError('[invalid feat_type] use smt_feat or vgg')

model: From str method of Model: fc1 is Linear(in_features=4096, out_features=1000, bias=True), fc2 is Linear(in_features=1000, out_features=300, bias=True)
model: From str method of Model: fc1 is Linear(in_features=4096, out_features=1000, bias=True), fc2 is Linear(in_features=1000, out_features=300, bias=True)


In [5]:
# prepair output dir
d_name = 'summe'
dataset_root = 'data/{}/'.format(d_name)
out_dir = 'results/{:}/{:}/'.format(d_name, feat_type)
print('save to: ', out_dir)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

save to:  results/summe/smt_feat/


In [6]:
# load dataset metadata
dataset = json.load(open(dataset_root + 'dataset.json'))
video_id = [d['videoID'] for d in dataset]

print('Video list:')
for vi in video_id:
    print('-', vi)

Video list:
- Cooking


In [7]:
# summarize video
v_id = 'Cooking'

with configuration.using_config('train', False):
    with chainer.no_backprop_mode():
        vsum = VSUM(v_id, model, dataset=d_name, seg_l=5)

_, frames, _ = vsum.summarizeRep(seg_l=5, weights=[1.0, 0.0])

# get 0/1 label for each frame
fps = vsum.dataset.data['fps']
fnum = vsum.dataset.data['fnum']
label = get_flabel(frames, fnum, fps, seg_l=5)

ロードした動画のID: Cooking
budget: 2
objectives:  [<function representativeness.<locals>.<lambda> at 0x7fb4c230c320>, <function uniformity.<locals>.<lambda> at 0x7fb4c230c050>]
selected:  [4, 73]
frames:  [['000075.jpg', '000090.jpg', '000104.jpg', '000119.jpg', '000134.jpg'], ['001100.jpg', '001115.jpg', '001129.jpg', '001144.jpg', '001159.jpg']]
gt_score:  [[0.0, 0.0, 0.058823529411764705, 0.058823529411764705, 0.058823529411764705], [0.35294117647058826, 0.11764705882352941, 0.058823529411764705, 0.0, 0.0]]


In [8]:
!pwd
!python script/summarize.py
# !python script/evaluate.py results/summe/smt_feat

/content/drive/My Drive/video_summarize_dsf
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
/usr/local/lib/python3.7/dist-packages/chainer/_environment_check.py:75: UserWarning: 
--------------------------------------------------------------------------------
CuPy (cupy-cuda111) version 9.4.0 may not be compatible with this version of Chainer.
Please consider installing the supported version by running:
  $ pip install 'cupy-cuda111>=7.7.0,<8.0.0'

See the following page for more details:
  https://docs.cupy.dev/en/latest/install.html
--------------------------------------------------------------------------------

  requirement=requirement, help=help))
Toi day 1
Toi day 2
resultの保存先: results/summe/smt_feat/
dataset:  [{'image': ['000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg', '000006.jpg', '000007.jpg', '000008.jpg', '000009.jpg', '000010.jpg', '000011.jpg', '000012.jpg', '000013.jpg', '000014.jpg', '000015.jpg', '000016.jpg', '000017.jpg', '000018.jpg', '000

In [ ]:
# write summarized video
from skvideo.io import vread, vwrite

# video_path = 'video_summarize_dsf/data/summe/%s.webm' % v_id
video_path = 'data/summe/video.mp4'

video_data = vread(video_path)
sum_vid = video_data[label.ravel().astype(np.bool), :,:,:]

print ('writing video to', 'sum_%s.mp4'%v_id)
vwrite('sum_%s.mp4'%v_id, sum_vid)